# Web Scrapping: Normativa en trámite fuera de palzo para observaciones (SUSESO)

El siguiente código realiza una revisión y extracción del código fuente de la página de la Superintendencia de Salud. Los requisitos para poder ejecutar este código son:

 - **Instalar librerías:**
  - pandas
  - scrapy
  - requests
  - datetime
  - wget
 - **Tener creadas las carpetas para almacenar los archivos.**
  - En el **código 17** está la ruta en donde se almacenan, la cual debe ser cambiada por quien ejecute el código para que referencie a su propio equipo.
  
---
 - **Fecha de creación: 09-10-2020 (IIC)**
 - **Fecha modificación: 14.10.2020 (IIC)**

## Importar librerías

In [1]:
import pandas as pd
from scrapy import Selector
import requests
from datetime import datetime, timedelta
import wget

## Cargar códigos html de las páginas a utilizar

In [2]:
html = requests.get('https://www.suseso.cl/612/w3-propertyvalue-34036.html').content

### Crear selectors para después buscar la información

In [3]:
sel = Selector(text = html)

## Búsqueda de información

### Lista de fechas

In [4]:
fechas = sel.xpath('//td[contains(@class,"fecha cid-512 pnid-524")]/text()').extract()

In [5]:
#Se elimina registro sin link de 2011
fechas.remove("13/09/2011")

### Lista Materia

In [6]:
materia = sel.xpath('//td[contains(@class,"materia cid-512 pnid-525")]/text()').extract()

In [7]:
#Se elimina registro sin link de 2011
materia.remove("Proyecto de Circular conjunta de las Superintendencias de Pensiones, Valores y Seguros y de Seguridad Social. Establece regulaciones comunes relativas a las obligaciones de las Administradoras de Fondos de Pensiones, Compañías de Seguros de Vida, Mutualidades de Empleadores de la Ley N° 16.744, Instituto de Seguridad Laboral e Instituto de Previsión Social, respecto de la determinación y pago de los beneficios relativos a la exención de la cotización legal de salud. Agrega nuevo Título XVI al Libro III del Compendio de Normas del Sistema de Pensiones de la Superintendencia de Pensiones.")

### Lista fecha para observaciones

In [8]:
fecha_obs = sel.xpath('//td[contains(@class,"fecha cid-512 pnid-555")]/text()').extract()

In [9]:
#Se elimina registro sin link de 2011
fecha_obs.remove("23/09/2011 ")

### Lista links

In [10]:
links = sel.xpath('//td[contains(@class,"normativa-consultar")]/span[contains(@class,"figure cid-512")]/a/@href').extract()

In [11]:
urls = []

for i in links:
    url = 'https://www.suseso.cl/612/'+i
    urls.append(url)

## Almacenar información obtenida y agregar formatos

In [12]:
df_norm = pd.DataFrame({
    'Fecha Publicación': fechas,
    'Materia' : materia,
    'Plazo Observación': fecha_obs,
    'Link': urls})

# Agregamos columna aux y la unimos con la columna nombre
df_norm['aux'] = df_norm.index.tolist()
df_norm['Nombre'] = df_norm['aux'].astype(str) + '_' + df_norm['Fecha Publicación'] + '_' + df_norm['Plazo Observación']
df_norm['Nombre'] = df_norm['Nombre'].str.replace('/','-')
df_norm = df_norm.drop(['aux'], axis=1)

# Limpiamos espacios en columnas de interés
df_norm['Fecha Publicación'] = df_norm['Fecha Publicación'].str.replace(" ","")
df_norm['Plazo Observación'] = df_norm['Plazo Observación'].str.replace(" ","")

# Agregamos formato fecha a la columna "Fecha Publicación" y "Plazo Observación"
df_norm['Fecha Publicación'] = pd.to_datetime(df_norm['Fecha Publicación'], format='%d/%m/%Y')
df_norm['Plazo Observación'] = pd.to_datetime(df_norm['Plazo Observación'], format='%d/%m/%Y')

## Filtrar novedades

In [13]:
novedad = datetime.now() - timedelta(30)

### Circulares internas

In [14]:
df_norm_novedad = df_norm.loc[(df_norm['Fecha Publicación'] >= novedad)]
df_norm_novedad

In [15]:
links_novedad = df_norm['Link']

In [16]:
nombre_archivo = df_norm['Nombre']

## Descargar novedades

### Rutas para almacenar la información

In [17]:
paths = []

for i in nombre_archivo:
    # La ruta escrita entre '' debe ser cambiada por quien ejecute el código.
    path = 'C:/projects/datalake/data/output/current/cambios_normativos/suseso/normativa_tramite/' + i
    paths.append(path)

### Descargar archivos

In [18]:
i = 0

for url in links_novedad:
    wget.download(url,paths[i]+'.pdf')
    i = i + 1

### Guardar DF históricos en Excel

In [19]:
nombre_archivo = 'Normativa_trámite(fuera_de_plazo)'
output_path = 'C:/projects/datalake/data/output/current/cambios_normativos/suseso/normativa_tramite/'

df_norm.to_csv(output_path + nombre_archivo + '.csv')